# Introduction to XGBoost Spark3.0 with GPU

Agaricus is an example of xgboost classifier for multiple classification. This notebook will show you how to load data, train the xgboost model and use this model to predict if a mushroom is "poisonous". Camparing to original XGBoost Spark code, there're only one API difference.

## Load libraries
First load some common libraries will be used by both GPU version and CPU version xgboost.

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}

Besides CPU version requires some extra libraries, such as:

```scala
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions._
```

## Set the dataset paths

In [2]:
// You need to update them to your real paths!
val trainPath = "/data/agaricus/csv/train/"
val evalPath  = "/data/agaricus/csv/test/"
val transPath = "/data/agaricus/csv/test/"

trainPath = /data/agaricus/csv/train/
evalPath = /data/agaricus/csv/test/
transPath = /data/agaricus/csv/test/


/data/agaricus/csv/test/

## Build the schema of the dataset

For agaricus example, the data has 126 dimensions, being named as "feature_0", "feature_1" ... "feature_125". The schema will be used to load data in the future.

In [3]:
val labelName = "label"
def columnNames(length: Int): List[String] =
  0.until(length).map(i => s"feature_$i").toList.+:(labelName)

def schema(length: Int): StructType =
  StructType(columnNames(length).map(n => StructField(n, DoubleType)))

val dataSchema = schema(126)

// Build the column name list for features.
val featureCols = dataSchema.filter(_.name != labelName).map(_.name)

labelName = label
dataSchema = StructType(StructField(label,DoubleType,true), StructField(feature_0,DoubleType,true), StructField(feature_1,DoubleType,true), StructField(feature_2,DoubleType,true), StructField(feature_3,DoubleType,true), StructField(feature_4,DoubleType,true), StructField(feature_5,DoubleType,true), StructField(feature_6,DoubleType,true), StructField(feature_7,DoubleType,true), StructField(feature_8,DoubleType,true), StructField(feature_9,DoubleType,true), StructField(feature_10,DoubleType,true), StructField(feature_11,DoubleType,true), StructField(feature_12,DoubleType,true), StructField(feature_13,DoubleType,true), StructFiel...


columnNames: (length: Int)List[String]
schema: (length: Int)org.apache.spark.sql.types.StructType


StructType(StructField(label,DoubleType,true), StructField(feature_0,DoubleType,true), StructField(feature_1,DoubleType,true), StructField(feature_2,DoubleType,true), StructField(feature_3,DoubleType,true), StructField(feature_4,DoubleType,true), StructField(feature_5,DoubleType,true), StructField(feature_6,DoubleType,true), StructField(feature_7,DoubleType,true), StructField(feature_8,DoubleType,true), StructField(feature_9,DoubleType,true), StructField(feature_10,DoubleType,true), StructField(feature_11,DoubleType,true), StructField(feature_12,DoubleType,true), StructField(feature_13,DoubleType,true), StructFiel...

## Create a new spark session and load data

A new spark session should be created to continue all the following spark operations.

NOTE: in this notebook, the dependency jars have been loaded when installing toree kernel. Alternatively the jars can be loaded into notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called just after a new spark session is created. Do it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("agaricus-GPU").getOrCreate
%AddJar file:/data/libs/cudf-XXX-cuda10.jar
%AddJar file:/data/libs/rapids-4-spark-XXX.jar
%AddJar file:/data/libs/xgboost4j_3.0-XXX.jar
%AddJar file:/data/libs/xgboost4j-spark_3.0-XXX.jar
// ...
```

##### Please note the new jar "rapids-4-spark-XXX.jar" is only needed for GPU version, you can not add it to dependence list for CPU version.

In [4]:
// Build the spark session and data reader as usual
val sparkSession = SparkSession.builder.appName("agaricus-gpu").getOrCreate
val dataReader = sparkSession.read.option("header", true).schema(dataSchema)

sparkSession = org.apache.spark.sql.SparkSession@a5529d7
dataReader = org.apache.spark.sql.DataFrameReader@6622bdb3


org.apache.spark.sql.DataFrameReader@6622bdb3

In [6]:
// load all the dataset
val trainSet = dataReader.csv(trainPath)
val evalSet  = dataReader.csv(evalPath)
val transSet = dataReader.csv(transPath)

trainSet = [label: double, feature_0: double ... 125 more fields]
evalSet = [label: double, feature_0: double ... 125 more fields]
transSet = [label: double, feature_0: double ... 125 more fields]


[label: double, feature_0: double ... 125 more fields]

## Set xgboost parameters and build a XGBoostClassifier

For CPU version, `num_workers` is recommended being equal to the number of CPU cores, while for GPU version, it should be set to the number of GPUs in Spark cluster.

Besides the `tree_method` for CPU version is also different from that for GPU version. Now only "gpu_hist" is supported for training on GPU.

```scala
// difference in parameters
  "num_workers" -> 12,
  "tree_method" -> "hist",
```

In [7]:
// build XGBoost classifier
val paramMap = Map(
  "eta" -> 0.1,
  "max_depth" -> 2,
  "num_workers" -> 1,
  "tree_method" -> "gpu_hist",
  "missing" -> 0.0,
  "num_round" -> 100
)

paramMap = Map(num_workers -> 1, max_depth -> 2, num_round -> 100, missing -> 0.0, tree_method -> gpu_hist, eta -> 0.1)


Map(num_workers -> 1, max_depth -> 2, num_round -> 100, missing -> 0.0, tree_method -> gpu_hist, eta -> 0.1)

Here comes the only API difference,`setFeaturesCol` in CPU version vs `setFeaturesCols` in GPU version.

In previous block, it said that CPU version needs `VectorAssembler` to assemble multiple feature columns into one column, because `setFeaturesCol` only accepts one feature column with the type of `vector`.

But `setFeaturesCols` supports multiple columns directly, so set the feautres column names directly to `XGBoostClassifier`. 

CPU version:

```scala
val xgbClassifier  = new XGBoostClassifier(paramMap)
  .setLabelCol(labelName)
  .setFeaturesCol("features")
```

In [8]:
val xgbClassifier  = new XGBoostClassifier(paramMap)
  .setLabelCol(labelName)
  // === diff ===
  .setFeaturesCols(featureCols)

xgbClassifier = xgbc_ed4347205fcc


xgbc_ed4347205fcc

## Benchmark and train
The object `benchmark` is used to compute the elapsed time of some operations.

Training with evaluation sets is also supported in 2 ways, the same as CPU version's behavior:

* Call API `setEvalSets` after initializing an XGBoostClassifier

```scala
xgbClassifier.setEvalSets(Map("eval" -> evalSet))

```

* Use parameter `eval_sets` when initializing an XGBoostClassifier

```scala
val paramMapWithEval = paramMap + ("eval_sets" -> Map("eval" -> evalSet))
val xgbClassifierWithEval = new XGBoostClassifier(paramMapWithEval)
```

Here chooses the API way to set evaluation sets.

In [9]:
xgbClassifier.setEvalSets(Map("eval" -> evalSet))

xgbc_ed4347205fcc

In [10]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("Elapsed time [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

defined object Benchmark


CPU version reqires an extra step before fitting data to classifier, using `VectorAssembler` to assemble all feature columns into one column. The following code snip shows how to do the vectorizing.

```scala
object Vectorize {
  def apply(df: DataFrame, featureNames: Seq[String], labelName: String): DataFrame = {
    val toFloat = df.schema.map(f => col(f.name).cast(FloatType))
    new VectorAssembler()
      .setInputCols(featureNames.toArray)
      .setOutputCol("features")
      .transform(df.select(toFloat:_*))
      .select(col("features"), col(labelName))
  }
}

trainSet = Vectorize(trainSet, featureCols, labelName)
evalSet = Vectorize(evalSet, featureCols, labelName)
transSet = Vectorize(transSet, featureCols, labelName)

```

Fortunately `VectorAssembler` is not needed for GPU version. Just fit the loaded data to XGBoostClassifier directly.

In [11]:
// start training
println("\n------ Training ------")
val (xgbClassificationModel, _) = Benchmark.time("train") {
  xgbClassifier.fit(trainSet)
}


------ Training ------
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.78, DMLC_TRACKER_PORT=9091, DMLC_NUM_WORKER=1}


xgbClassificationModel = xgbc_ed4347205fcc


Elapsed time [train]: 17.184s


xgbc_ed4347205fcc

## Transformation and evaluation
Here uses `transSet` to evaluate our model and prints some useful columns to show our prediction result. After that `MulticlassClassificationEvaluator` is used to calculate an overall accuracy of our predictions.

In [12]:
// start transform
println("\n------ Transforming ------")
val (results, _) = Benchmark.time("transform") {
  val ret = xgbClassificationModel.transform(transSet).cache()
  ret.foreachPartition((_: Iterator[_]) => ())
  ret
}
results.select(labelName, "rawPrediction", "probability", "prediction").show(10)

println("\n------Accuracy of Evaluation------")
val evaluator = new MulticlassClassificationEvaluator()
evaluator.setLabelCol(labelName)
val accuracy = evaluator.evaluate(results)

println(s"accuracy == $accuracy")


------ Transforming ------
Elapsed time [transform]: 2.128s
+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0080436170101...|[0.99195638298988...|       0.0|
|  0.0|[-0.0080436170101...|[0.99195638298988...|       0.0|
|  0.0|[-0.1416745483875...|[0.85832545161247...|       0.0|
|  0.0|[-0.0747678875923...|[0.92523211240768...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0145334601402...|[0.98546653985977...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0457237958908...|[0.95427620410919...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 10 rows


------Accuracy of Evaluation------
accuracy == 0.998757706

results = [label: double, feature_0: double ... 128 more fields]
evaluator = MulticlassClassificationEvaluator: uid=mcEval_f74396de0ab9, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15
accuracy = 0.9987577063864658


0.9987577063864658

## Save the model to disk and load model
Save the model to disk and then load it to memory. After that use the loaded model to do a new prediction.

In [13]:
xgbClassificationModel.write.overwrite.save("/data/model/agaricus")

val modelFromDisk = XGBoostClassificationModel.load("/data/model/agaricus")
val (results2, _) = Benchmark.time("transform2") {
  modelFromDisk.transform(transSet)
}
results2.select(labelName, "rawPrediction", "probability", "prediction").show(10)

Elapsed time [transform2]: 0.076s


modelFromDisk = xgbc_ed4347205fcc
results2 = [label: double, feature_0: double ... 128 more fields]


+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0080436170101...|[0.99195638298988...|       0.0|
|  0.0|[-0.0080436170101...|[0.99195638298988...|       0.0|
|  0.0|[-0.1416745483875...|[0.85832545161247...|       0.0|
|  0.0|[-0.0747678875923...|[0.92523211240768...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0145334601402...|[0.98546653985977...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
|  0.0|[-0.0457237958908...|[0.95427620410919...|       0.0|
|  1.0|[-0.9667758941650...|[0.03322410583496...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 10 rows



[label: double, feature_0: double ... 128 more fields]

In [14]:
sparkSession.close()